In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
class BaseTransformer:
    def __init__(self, sparkSession):
        self.spark = sparkSession
    def readData(self, path, format="json", schema=None):
        reader=self.spark.read.format(format)
        if schema:
            reader.schema(schema)
        df=reader.load(path)
        return df
    def writeData(self, df, path, format="json", dbPath=None, checkpointPath=None, mode="append", streaming=False):
        if streaming:
            writer = (
                df.writeStream
                .format(format)
                .option("checkpointLocation", checkpointPath)
                .outputMode(mode)
            )
            query = writer.toTable(dbPath)
            query.awaitTermination()
        else:
            (
                df.write
                .format(format)
                .mode(mode)
                .save(path)
            )
    def showShape(self, df):
        return (len(df.columns), df.count())

    def convertTimestamp(self, df, colName="created_utc", newCol="createdDate"):
        return df.withColumn(
            newCol,
            F.from_unixtime(F.col(colName)).cast("timestamp")
        )

    def markAuthorDeleted(self, df, authorCol="author", nameAuthor="author_fullname", newCol="accDeleted"):
        return df.withColumn(
            newCol,
            (F.col(authorCol) == "[deleted]") & F.col(nameAuthor).isNull()
        )

    def markBodyRemoved(self, df, body="selftext", newCol="postDeleted"):
        return df.withColumn(
            newCol,
            F.col(body) == "[removed]"
        )

In [3]:
class SubmissionTransformer(BaseTransformer):
    def __init__(self, sparkSession):
        super().__init__(sparkSession)

    def markSpamPost(self, df, titleCol="title", urlCol="url", domainCol="domain", subredditCol="subreddit", newCol="isPostSpam"):
        words = F.split(F.lower(F.col(titleCol)), " ")
        numMatched = F.size(
        F.expr(f"filter(split(lower({titleCol}), ' '), word -> instr(lower({urlCol}), word) > 0)")
        )        
        totalWord = F.size(words)
        matchPercent = (numMatched * 100) / totalWord
        isPostSpam = (matchPercent >= 0.6) & (
        F.col(domainCol) != F.concat(F.lit("self."), F.col(subredditCol))
        )
        
        return df.withColumn("matchPercent", matchPercent).withColumn(newCol, isPostSpam)        

In [4]:
spark = SparkSession.builder.appName("SubmissionTransformer").getOrCreate()

25/08/10 14:16:07 WARN Utils: Your hostname, Acer resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp4s0)
25/08/10 14:16:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/10 14:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
submission = SubmissionTransformer(spark)

In [6]:
path = "../data/RS_reddit.jsonl"
rSubmissionBronzeSchema = StructType([
    StructField("id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("selftext", StringType(), True),
    StructField("url", StringType(), True),
    StructField("permalink", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("post_hint", StringType(), True),
    StructField("author", StringType(), True),
    StructField("author_fullname", StringType(), True),
    StructField("created_utc", LongType(), True),
    StructField("subreddit", StringType(), True),
    StructField("subreddit_id", StringType(), True),
    StructField("subreddit_name_prefixed", StringType(), True),
    StructField("subreddit_type", StringType(), True),
    StructField("subreddit_subscribers", IntegerType(), True),
    StructField("score", IntegerType(), True),
    StructField("num_comments", IntegerType(), True),
    StructField("total_awards_received", IntegerType(), True),
    StructField("edited", BooleanType(), True),
    StructField("locked", BooleanType(), True),
    StructField("spoiler", BooleanType(), True),
    StructField("over_18", BooleanType(), True),
    StructField("stickied", BooleanType(), True),   
    StructField("retrieved_on", LongType(), True),
    StructField("is_original_content", BooleanType(), True),
    StructField("link_flair_text", StringType(), True)
])


In [7]:
df = submission.readData(path, schema=rSubmissionBronzeSchema)

In [15]:
submission.showShape(df)

(31, 7306)

In [14]:
df.show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+---------------+-----------+-----------+------------+-----------------------+--------------+---------------------+-----+------------+---------------------+------+------+-------+-------+--------+------------+-------------------+--------------------+-------------------+----------+-----------+------------------+----------+
|     id|               title|            selftext|                 url|           permalink|              domain|post_hint|              author|author_fullname|created_utc|  subreddit|subreddit_id|subreddit_name_prefixed|subreddit_type|subreddit_subscribers|score|num_comments|total_awards_received|edited|locked|spoiler|over_18|stickied|retrieved_on|is_original_content|     link_flair_text|        createdDate|accDeleted|postDeleted|      matchPercent|isPostSpam|
+-------+--------------------+--------------------+---------------

In [10]:
df = submission.convertTimestamp(df)

In [11]:
df = submission.markAuthorDeleted(df)

In [12]:
df = submission.markBodyRemoved(df)

In [13]:
df = submission.markSpamPost(df)